## Import modules

In [1]:
import sys
import time
import json
import pickle

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import uproot
import concurrent.futures

from XRootD import client
from XRootD.client.flags import DirListFlags, StatInfoFlags, OpenFlags, MkDirFlags, QueryCode

Import local classes from utils

In [2]:
%load_ext autoreload
%autoreload 1
%aimport utils.ObjectExtractor
%aimport utils.PlotMaker
%aimport utils.HistogramContainer
%aimport utils.HistogramCalculator
OE = utils.ObjectExtractor
PM = utils.PlotMaker
HCont = utils.HistogramContainer
HCalc = utils.HistogramCalculator

## Global initialization

In [3]:
print(sys.version_info)
# num_cores = multiprocessing.cpu_count()
# print(num_cores)


executor = concurrent.futures.ThreadPoolExecutor(48)

plt.style.use('default')
plt.rcParams['grid.linestyle'] = ':'
plt.rcParams.update({'font.size': 10})

numCuts = np.arange(0,6)

branch_path = 'SREffi_gbm'

labels = [ f'cut{cut}' for cut in numCuts ]
cut_descriptions = [
    'cut1: MET/MHT trigger fired (120 GeV)',
    'cut2: j1 pT > 120 GeV, <= 2j w/ pT > 30 GeV',
    'cut3: mu1 pT > 5 GeV, 0.1 < |dxy| < 700 cm',
    'cut4: mu2 pT > 5 GeV, 0.1 < |dxy| < 700 cm',
    r'cut5: $|\Delta\Phi$(MET, mu pair)| < 0.4',
]

sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)


In [4]:
all_plot_vars = ['metpt', 'jetpt','metjetphi', 'metmuphi', 'leadingmupt', 'subleadingmupt', 'recodr', 'recovertex']
plot_vars_metjet = all_plot_vars[0:4] #['metpt', 'jetpt', 'metjetphi', 'metmuphi']
plot_vars_muons = all_plot_vars[4:8] #['leadingmupt', 'subleadingmupt', 'recodr', 'recovertex']
cutflow_vars = ['cutflow_incl', 'cutflow_excl']
all_plot_xlabels = [
    'MET [GeV]', 'Leading jet pT [GeV]', '$\Delta\Phi$(MET, jet)', '$\Delta\Phi$(MET, di-muon)',
    'Leading muon pT [GeV]', 'Subleading muon pT [GeV]', 'dR(muons)', 'Di-muon vertex [cm]']

In [5]:
histos = {}
all_bins = {}
for plot_var in all_plot_vars:
    histos[plot_var] = {}
    all_bins[plot_var] = 60
histos['cutflow_incl'] = {}
histos['cutflow_excl'] = {}
histos['sumgenwgt'] = {}

## Process signal

In [6]:
## new signal input
with open('config/sig.json') as sigs_json_file:
    sigs = json.load(sigs_json_file)

xrdfs = client.FileSystem("root://cmseos.fnal.gov/")

redirector = 'root://cmsxrootd.fnal.gov'
sig_base_dir = '/store/group/lpcmetx/iDM/Ntuples/2018/signal/track_quality/iDM_2018_MC'
files = {}

for sig, properties in sigs.items():
    files[sig] = []
    status, listing = xrdfs.dirlist(f'{sig_base_dir}/{properties["dir"]}', DirListFlags.STAT)
    for file in listing:
        if '.root' in file.name:
            files[sig].append(f'{redirector}/{sig_base_dir}/{properties["dir"]}/{file.name}')
num_files_total = np.sum(np.array([len(files[i]) for i in files]))
print(num_files_total)
[(i, len(files[i])) for i in files]

724


[('Mchi-60p0_dMchi-20p0_ctau-1', 229), ('Mchi-5p25_dMchi-0p5_ctau-1000', 495)]

In [7]:
%%time

MAX_FILES=None # To load all possible files
# MAX_FILES=1 # For testing

### Initialize empty dicts of histograms 
# histos = {}
# all_bins = {}
# for plot_var in all_plot_vars:
#     histos[plot_var] = {}
#     all_bins[plot_var] = 60
# histos['cutflow_incl'] = {}
# histos['cutflow_excl'] = {}
# histos['sumgenwgt'] = {}

global_file_counter = 1

for sig in sigs:
    
    print(f'Processing signal {sig} ({(list(sigs.keys())).index(sig)+1}/{len(sigs)})')
    
    ### Initialize histograms as empty HistogramContainers
    for plot_var in all_plot_vars:
        histos[plot_var][sig] = HCont.HistogramContainer(all_bins[plot_var])
    histos['cutflow_incl'][sig] = np.zeros(len(numCuts))
    histos['cutflow_excl'][sig] = np.zeros(len(numCuts))
    histos['sumgenwgt'][sig] = 0.0
    
    ### Load data
    file_counter = 1
    for file in files[sig][slice(0,MAX_FILES)]:
        
        if file_counter % 10 == 1:
            print(f'Reading file {file_counter} of {len(files[sig])},'
                  f' global {global_file_counter} of {num_files_total}'
                  f' ({100*(global_file_counter-1)/num_files_total:.2f}%)')
            with open('histos_temp.dat', 'wb') as histos_file:
                pickle.dump(histos, histos_file)
        file_counter += 1
        global_file_counter += 1
        
        ### Open ROOT file and get tree
        tree = uproot.open(file)[branch_path + '/cutsTree']
        
        ### Make pandas dataframes and create all objects that will be passed to histo functions
        obj_extractor = OE.ObjectExtractor(tree)
        objects = obj_extractor.get_all()
            
        ## Add to sum of genwgts
        histos['sumgenwgt'][sig] += np.sum(objects['genwgt'])
        
        ### Calculate histograms and cutflows
        histo_maker = HCalc.HistogramCalculator(objects, sig)
            
        ### Cutflows
        incl, excl = histo_maker.cutflows()
        histos['cutflow_incl'][sig] += incl
        histos['cutflow_excl'][sig] += excl
        
        ### Histograms
        for plot_var in all_plot_vars:
            new_hist = eval(f'histo_maker.{plot_var}()')
            histos[plot_var][sig] += new_hist

Processing signal Mchi-60p0_dMchi-20p0_ctau-1 (1/2)
Reading file 1 of 229, global 1 of 724 (0.00%)
Reading file 11 of 229, global 11 of 724 (1.38%)
Reading file 21 of 229, global 21 of 724 (2.76%)
Reading file 31 of 229, global 31 of 724 (4.14%)
Reading file 41 of 229, global 41 of 724 (5.52%)
Reading file 51 of 229, global 51 of 724 (6.91%)
Reading file 61 of 229, global 61 of 724 (8.29%)
Reading file 71 of 229, global 71 of 724 (9.67%)
Reading file 81 of 229, global 81 of 724 (11.05%)
Reading file 91 of 229, global 91 of 724 (12.43%)
Reading file 101 of 229, global 101 of 724 (13.81%)
Reading file 111 of 229, global 111 of 724 (15.19%)
Reading file 121 of 229, global 121 of 724 (16.57%)
Reading file 131 of 229, global 131 of 724 (17.96%)
Reading file 141 of 229, global 141 of 724 (19.34%)
Reading file 151 of 229, global 151 of 724 (20.72%)
Reading file 161 of 229, global 161 of 724 (22.10%)
Reading file 171 of 229, global 171 of 724 (23.48%)
Reading file 181 of 229, global 181 of 724

In [8]:
luminosity = 59.97 * 1000 # 1/pb
for sig, properties in sigs.items():
    properties['weight'] = luminosity * properties['xsec'] / histos['sumgenwgt'][sig]
#     except KeyError:
#         properties['weight'] = 1
for sig, properties in sigs.items():
    try:
        print(sig, luminosity * properties['xsec'] / histos['sumgenwgt'][sig], histos['sumgenwgt'][sig])
    except KeyError: pass
    
with open('histos_signal_objects_gbm.dat', 'wb') as histos_file:
    pickle.dump(histos, histos_file)

Mchi-60p0_dMchi-20p0_ctau-1 5241.240589198036 61100.0
Mchi-5p25_dMchi-0p5_ctau-1000 5135.092923688726 62363.0


In [ ]:
#masses = [('60p0','20p0'),('6p0','2p0'),('52p5','5p0'),('5p25','0p5')]
#def print_masses(mass):
#    return f'({float(mass[0].replace("p","."))-float(mass[1].replace("p","."))/2}, ' + \
#           f'{float(mass[0].replace("p","."))+float(mass[1].replace("p","."))/2}) GeV'
#    
#mchis = dict([(mass[0], print_masses(mass)) for mass in masses])
#ctaus = [10]#, 10, 100, 1000]
#
#
#base_dir = '../Firefighter/washAOD/SROptimization/'
#def filename(Mchi, dMchi, ctau): 
#    return base_dir + f'Mchi-{Mchi}_dMchi-{dMchi}_ctau-{ctau}.root'

In [ ]:
#trees_gbm = dict()
#gen_info_gbm = dict()

In [ ]:
#num_params = len(masses)*len(ctaus)
#count_param = 1
#
#for (Mchi, dMchi) in masses:
#    trees_gbm[Mchi] = dict()
#    gen_info_gbm[Mchi] = dict()
#    for ctau in ctaus:
#        gen_info_gbm[Mchi][ctau] = uproot.open(filename(Mchi, dMchi, ctau))['GEN/gen']#.pandas.df(flatten=False)
#        trees_gbm[Mchi][ctau] = uproot.open(filename(Mchi, dMchi, ctau))[branch_path + f'/cutsTree']#.pandas.df(flatten=False)
#        print(f'{count_param} of {num_params}: ' + filename(Mchi, dMchi, ctau))
#        count_param += 1

In [ ]:
#histos_signal = {}
#for plot_var in all_plot_vars:
#    histos_signal[plot_var] = {}
#for plot_var in cutflow_vars:
#    histos_signal[plot_var] = {}
#    
#for mchi in mchis:
#    for plot_var in all_plot_vars:
#        histos_signal[plot_var][mchi] = HCont.HistogramContainer(all_bins[plot_var])
#    histos_signal['cutflow_incl'][mchi] = np.zeros(len(numCuts))
#    histos_signal['cutflow_excl'][mchi] = np.zeros(len(numCuts))
#        
#    ### Make pandas dataframes and create all objects that will be passed to histo functions
#    obj_extractor = OE.ObjectExtractor(trees_gbm[mchi][ctau], mchi)
#    objects = obj_extractor.get_all()
#
#    ### Calculate histograms and cutflows
#    histo_computer = HCalc.HistogramCalculator(objects, mchi)
#
#    ### Cutflows
#    incl, excl = histo_computer.cutflows()
#    histos_signal['cutflow_incl'][mchi] += incl
#    histos_signal['cutflow_excl'][mchi] += excl
#
#    ### Histograms
#    for plot_var in all_plot_vars:
#        new_hist = eval(f'histo_computer.{plot_var}()')
#        histos_signal[plot_var][mchi] += new_hist

In [ ]:
#with open('histos_signal_objects_gbm.dat', 'wb') as histos_file:
#    pickle.dump(histos_signal, histos_file)

## Process backgrounds

In [ ]:
with open('config/bkgs3.json') as bkgs_json_file:
    bkgs = json.load(bkgs_json_file)

In [ ]:
xrdfs = client.FileSystem("root://cmseos.fnal.gov/")

redirector = 'root://cmsxrootd.fnal.gov'
bkg_base_dir = '/store/group/lpcmetx/iDM/Ntuples/2018/backgrounds'
files = {}

for bkg, properties in bkgs.items():
    files[bkg] = []
    status, listing = xrdfs.dirlist(f'{bkg_base_dir}/{properties["dir"]}', DirListFlags.STAT)
    for file in listing:
        if '.root' in file.name:
            files[bkg].append(f'{redirector}/{bkg_base_dir}/{properties["dir"]}/{file.name}')

In [ ]:
num_files_total = np.sum(np.array([len(files[i]) for i in files]))
print(num_files_total)
[(i, len(files[i])) for i in files]

In [ ]:
%%time

MAX_FILES=None # To load all possible files
# MAX_FILES=1 # For testing

### Initialize empty dicts of histograms 
# histos = {}
# all_bins = {}
# for plot_var in all_plot_vars:
#     histos[plot_var] = {}
#     all_bins[plot_var] = 60
# histos['cutflow_incl'] = {}
# histos['cutflow_excl'] = {}
# histos['sumgenwgt'] = {}

global_file_counter = 1

for bkg in bkgs:
    
    print(f'Processing background {bkg} ({(list(bkgs.keys())).index(bkg)+1}/{len(bkgs)})')
    
    ### Initialize histograms as empty HistogramContainers
    for plot_var in all_plot_vars:
        histos[plot_var][bkg] = HCont.HistogramContainer(all_bins[plot_var])
    histos['cutflow_incl'][bkg] = np.zeros(len(numCuts))
    histos['cutflow_excl'][bkg] = np.zeros(len(numCuts))
    histos['sumgenwgt'][bkg] = 0.0
    
    ### Load data
    file_counter = 1
    for file in files[bkg][slice(0,MAX_FILES)]:
        
        if file_counter % 10 == 1:
            print(f'Reading file {file_counter} of {len(files[bkg])},'
                  f' global {global_file_counter} of {num_files_total}'
                  f' ({100*(global_file_counter-1)/num_files_total:.2f}%)')
            with open('histos_temp.dat', 'wb') as histos_file:
                pickle.dump(histos, histos_file)
        file_counter += 1
        global_file_counter += 1
        
        ### Open ROOT file and get tree
        tree = uproot.open(file)[branch_path + '/cutsTree']
        
        ### Make pandas dataframes and create all objects that will be passed to histo functions
        obj_extractor = OE.ObjectExtractor(tree)
        objects = obj_extractor.get_all()
            
        ## Add to sum of genwgts
        histos['sumgenwgt'][bkg] += np.sum(objects['genwgt'])
        
        ### Calculate histograms and cutflows
        histo_maker = HCalc.HistogramCalculator(objects, bkg)
            
        ### Cutflows
        incl, excl = histo_maker.cutflows()
        histos['cutflow_incl'][bkg] += incl
        histos['cutflow_excl'][bkg] += excl
        
        ### Histograms
        for plot_var in all_plot_vars:
            new_hist = eval(f'histo_maker.{plot_var}()')
            histos[plot_var][bkg] += new_hist

In [ ]:
luminosity = 59.97 * 1000 # 1/pb
for bkg, properties in bkgs.items():
    properties['weight'] = luminosity * properties['xsec'] / histos['sumgenwgt'][bkg]
#     except KeyError:
#         properties['weight'] = 1

In [ ]:
for bkg, properties in bkgs.items():
    try:
        print(bkg, luminosity * properties['xsec'] / histos['sumgenwgt'][bkg], histos['sumgenwgt'][bkg])
    except KeyError: pass

In [ ]:
with open('histos_bkgs_objects_gbm.dat', 'wb') as histos_file:
    pickle.dump(histos, histos_file)

In [ ]:
cutFlowInclGrp = {}
for grp in bkg_grps:
    if '60p0' in grp or '5p25' in grp or '52p5' in grp or '6p0' in grp: continue
    for bkg in bkg_grps[grp]:
        if grp in cutFlowInclGrp.keys():
            cutFlowInclGrp[grp] += histos['cutflow_incl'][bkg].astype(int)
        else:
            cutFlowInclGrp[grp] = histos['cutflow_incl'][bkg].astype(int)

pd.DataFrame.from_dict(cutFlowInclGrp)

In [ ]:
cutFlowInclGrp2 = {}
for grp in bkg_grps:
#     if '60p0' in grp or '5p25' in grp or '52p5' in grp or '6p0' in grp: continue
    for bkg in bkg_grps[grp]:
        if grp in cutFlowInclGrp2.keys():
            try:
                cutFlowInclGrp2[grp] += (histos['cutflow_incl'][bkg]*bkgs[bkg]['weight']).astype(int)
            except KeyError: pass
        else:
            try:
                cutFlowInclGrp2[grp] = (histos['cutflow_incl'][bkg]*bkgs[bkg]['weight']).astype(int)
            except KeyError: pass


pd.DataFrame.from_dict(cutFlowInclGrp2)